In [1]:
from typing import Dict

import requests
from tqdm import tqdm

In [2]:
AUTHORIZATION_CODE = "MML66LDKRB03H7RRB4ENVO63CUP55NJPTFDVJ79881C5ACP6L79QH9DCV1IDP972"
REDIRECT_URI = "https://www.alfabank.by"
USER_AGENT = "AlfaHRService (alfa-hr-service@alfabank.by)"
CLIENT_ID = "PC1JCC77M8707UGQ4NPQK31P7CT4T8MK2HL61QABANE9J7POHS1GDNTC78SRJMCB"
CLIENT_SECRET = "PJCS60M220HSBI0H1TKFRCS7PVOCD2QF69AKROHM2LVIRPHOFGOV06NBHPVN2H4U"
APP_ACCESS_TOKEN = "APPLS8AOGPI03E8OK9PKUABDT255LDGVSVQPLORCB4DC4313L3SUVQ9G98MB8J9F"
TOKEN_URL = "https://api.hh.ru/token"
REFRESH_TOKEN = "USERK1OTPDQREP2MT266HLTJNJTN97IR22SR4ECS55SQBD8DUI1UI0Q3S6MLH8GT"
ACCESS_TOKEN = "USERIBM3S8OHEFTBLSG29JV1R2QOTNGRCV7IGI4R76K597MFIMVUJ9JR58LO8K86"
EXPIRES_IN = 1209599


def exchange_code_for_token() -> Dict[str, any]:
    """
    ШАГ 1: Обменять код авторизации на access_token и refresh_token

    Args:
        authorization_code: Код, полученный после авторизации пользователя

    Returns:
        Словарь с токенами или ошибкой
    """
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "HH-User-Agent": USER_AGENT,
    }

    data = {
        "grant_type": "authorization_code",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "code": AUTHORIZATION_CODE,
        "redirect_uri": REDIRECT_URI,
    }

    print("🔄 Отправляю запрос на получение токена...")
    response = requests.post(TOKEN_URL, headers=headers, data=data, timeout=10)
    response.raise_for_status()

    token_data = response.json()
    return token_data


def refresh_access_token() -> Dict[str, any]:
    """
    ШАГ 2: Обновить access_token с помощью refresh_token

    Returns:
        Словарь с новыми токенами или ошибкой
    """
    if not REFRESH_TOKEN:
        print("❌ Нет refresh_token для обновления")
        return {"success": False, "error": "Refresh token not available"}

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "HH-User-Agent": USER_AGENT,
    }

    data = {
        "grant_type": "refresh_token",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "refresh_token": REFRESH_TOKEN,
    }

    try:
        print("🔄 Отправляю запрос на обновление токена...")
        response = requests.post(TOKEN_URL, headers=headers, data=data, timeout=10)
        response.raise_for_status()

        token_data = response.json()

        ACCESS_TOKEN = token_data.get("access_token")
        new_refresh = token_data.get("refresh_token")
        if new_refresh:
            REFRESH_TOKEN = new_refresh
            EXPIRES_IN = token_data.get("expires_in")

        print("✅ Токен успешно обновлен!\n")
        return {
            "success": True,
            "access_token": ACCESS_TOKEN,
            "refresh_token": REFRESH_TOKEN,
            "token_type": "bearer",
            "expires_in": EXPIRES_IN,
        }

    except requests.exceptions.HTTPError:
        error_data = response.json() if response.content else {}
        print(f"❌ Ошибка обновления токена: HTTP {response.status_code}")
        return {
            "success": False,
            "status_code": response.status_code,
            "error": error_data,
        }
    except requests.exceptions.RequestException as e:
        print(f"❌ Ошибка запроса: {e}")
        return {"success": False, "error": str(e)}

In [15]:
token_data = exchange_code_for_token()

🔄 Отправляю запрос на получение токена...


In [ ]:
number_of_pages = 2
headers = {
    "HH-User-Agent": "AlfaHRService (alfa-hr-service@alfabank.by)",
    "Authorization": f"Bearer {ACCESS_TOKEN}",
}

job = "Python developer"
areas = [16]
data = []
for area in areas:
    for i in tqdm(range(number_of_pages)):
        url = "https://api.hh.ru/resumes"
        if area == 16:
            host = "rabota.by"
        else:
            host = "hh.ru"
        # par = {
        #     "host": host,
        #     "text": job,
        #     "text.logic": "all",
        #     "text.field": "title", # title, skills, experience_position
        #     "text.period": "",
        #     "experience": "between3And6", # noExperience, between1And3, between3And6, moreThan6
        #     "text": "junior",
        #     "text.logic": "except",
        #     "text.field": "title", # title, skills, experience_position
        #     "text.period": "",
        #     "period": 30,
        #     "area": area,
        #     "per_page": "10",
        #     "page": i,
        # }
        par = [
            ("host", host),
            ("text", job),
            ("text.logic", "all"),
            ("text.field", "title"),  # title, skills, experience_position
            ("text.period", ""),
            (
                "experience",
                "between3And6",
            ),  # noExperience, between1And3, between3And6, moreThan6
            ("text", "junior"),
            ("text.logic", "except"),
            ("text.field", "title"),  # title, skills, experience_position
            ("text.period", ""),
            ("period", 30),
            ("area", area),
            ("per_page", "10"),
            ("page", i),
        ]
        r = requests.get(url, params=par, headers=headers)
        e = r.json()
        data.append(e)

100%|██████████| 2/2 [00:03<00:00,  1.63s/it]


In [34]:
data

[{'items': [{'last_name': None,
    'first_name': None,
    'middle_name': None,
    'title': 'Python-разработчик',
    'created_at': '2025-07-02T19:05:25+0300',
    'updated_at': '2026-02-10T11:44:05+0300',
    'area': {'id': '1002',
     'name': 'Минск',
     'url': 'https://api.hh.ru/areas/1002?host=rabota.by'},
    'age': 20,
    'gender': {'id': 'male', 'name': 'Мужской'},
    'salary': {'amount': 200000, 'currency': 'RUR'},
    'photo': None,
    'total_experience': {'months': 42},
    'certificate': [],
    'owner': {'id': '62341778',
     'comments': {'url': 'https://api.hh.ru/applicant_comments/62341778?host=rabota.by',
      'counters': {'total': 0}}},
    'can_view_full_info': False,
    'negotiations_history': {'url': 'https://api.hh.ru/resumes/99f65cb1000f110fc20008274a4b51716e6444/negotiations_history?host=rabota.by'},
    'hidden_fields': [],
    'actions': {'download': {'pdf': {'url': 'https://api.hh.ru/resumes/99f65cb1000f110fc20008274a4b51716e6444/download/Python-%D1%